In [ ]:
# prompt: connect to google drive

from google.colab import drive
drive.mount('/content/drive')

# import os
# path = '/content/drive/Othercomputers/My Laptop/Google Drive/Study/MS_PFW/GRAWork/potluck'
# os.chdir(path)

Mounted at /content/drive


In [ ]:
!pwd

/content


Download MIT_BIG Database from `wget -r -N -c -np https://physionet.org/files/mitdb/1.0.0/`

Using Python 3.12.4

Preprocessing help taken from https://github.com/physhik/ecg-mit-bih/tree/master/src

In [ ]:
# !wget -r -N -c -np https://physionet.org/files/mitdb/1.0.0/

In [ ]:
# !wget -r -N -c -np https://physionet.org/files/nsrdb/1.0.0/

In [ ]:
# !mv /content/physionet.org/files/* /content/drive/MyDrive/mitdb/

mv: cannot stat '/content/physionet.org/files/*': No such file or directory


In [ ]:
# prompt: pip install all requirements

!pip install wfdb
!pip install biosppy
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install tqdm
!pip install deepdish


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.0/405.0 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
import wfdb
import os
from glob import glob
import matplotlib.pyplot as plt
from wfdb import processing
import numpy as np
from scipy import signal
from scipy.signal import find_peaks
import neurokit2 as nk
import biosppy
from biosppy.signals import ecg
from sklearn import preprocessing
from tqdm import tqdm
import deepdish as dd
import csv

In [ ]:

classes = ['N','V','/','A','F','~','L','R','f','j','E','a','J','Q','e','S']
Nclass = len(classes)

In [ ]:
def add_noise(input_size):
  noises = dict()
  noises["trainset"] = []
  noises["testset"] = []
  try:
    testlabel = list(csv.reader(open('training2017/REFERENCE.csv')))
  except:
        cmd = "curl -O https://archive.physionet.org/challenge/2017/training2017.zip"
        os.system(cmd)
        os.system("unzip training2017.zip")
        testlabel = list(csv.reader(open('training2017/REFERENCE.csv')))
  for i, label in enumerate(testlabel):
    if label[1] == '~':
      filename = 'training2017/'+ label[0] + '.mat'
      from scipy.io import loadmat
      noise = loadmat(filename)
      noise = noise['val']
      _, size = noise.shape
      noise = noise.reshape(size,)
      noise = np.nan_to_num(noise) # removing NaNs and Infs
      from scipy.signal import resample
      noise= resample(noise, int(len(noise) * 360 / 300) ) # resample to match the data sampling rate 360(mit), 300(cinc)
      from sklearn import preprocessing
      noise = preprocessing.scale(noise)
      noise = noise/1000*6 # rough normalize, to be improved
      from scipy.signal import find_peaks
      peaks, _ = find_peaks(noise, distance=150)
      choices = 10 # 256*10 from 9000
      picked_peaks = np.random.choice(peaks, choices, replace=False)
      for j, peak in enumerate(picked_peaks):
        if peak > input_size//2 and peak < len(noise) - input_size//2:
            start,end  = peak-input_size//2, peak+input_size//2
            if i > len(testlabel)/6:
              noises["trainset"].append(noise[start:end].tolist())
            else:
                noises["testset"].append(noise[start:end].tolist())
  return noises

In [ ]:
def dataSaver(record_name, directory, features, zf=0):
  datadict, labeldict  = dict(), dict()

  # for feature in features:
  #   datadict[feature] = list()
  #   labeldict[feature] = list()

  def dataprocess(zf, dis=150, input_size=256):
    record_details = os.path.join(directory, record_name)
    record = wfdb.rdrecord(record_details, smooth_frames = True)
    available_features = record.sig_name

    for i, feature in enumerate(available_features):
        if feature not in datadict:
            datadict[feature] = list()
        if feature not in labeldict:
            labeldict[feature] = list()

    if zf == 0:
        zf = len(record.p_signal[:, 0])

    signals0 = preprocessing.scale(np.nan_to_num(record.p_signal[:,0][:zf])).tolist()
    signals1 = preprocessing.scale(np.nan_to_num(record.p_signal[:,1][:zf])).tolist()
    peaks, _ = find_peaks(signals0, distance=dis)
    feature0, feature1 = record.sig_name[0], record.sig_name[1]
    print(feature0, feature1)

    global lappend0, lappend1, dappend0, dappend1
    lappend0 = labeldict[feature0].append
    lappend1 = labeldict[feature1].append
    dappend0 = datadict[feature0].append
    dappend1 = datadict[feature1].append

    for peak in tqdm(peaks[1:-1]):
        start, end =  peak-input_size//2 , peak+input_size//2
        ann = wfdb.rdann(record_details, extension='atr', sampfrom=start, sampto=end, return_label_elements=['symbol'])

        annSymbol = ann.symbol
        if len(annSymbol) == 1 and (annSymbol[0] in classes) and (annSymbol[0] != "N" or np.random.random()<0.15):
            y = [0]*Nclass
            y[classes.index(annSymbol[0])] = 1
            lappend0(y)
            lappend1(y)
            dappend0(signals0[start:end])
            dappend1(signals1[start:end])

  dataprocess(zf=zf)
  for feature, data in datadict.items():
    print(f"Data length for {feature}: {len(data)}")
  noises = add_noise(input_size=256)
  for feature in features:
    if feature in datadict.keys():
      d = np.array(datadict[feature])
      if len(d) > 15*10**3:
          n = np.array(noises["trainset"])
      else:
          n = np.array(noises["testset"])
      datadict[feature]=np.concatenate((d,n))
      size, _  = n.shape
      l = np.array(labeldict[feature])
      noise_label = [0]*Nclass
      noise_label[-1] = 1

      noise_label = np.array([noise_label] * size)
      labeldict[feature] = np.concatenate((l, noise_label))

  return datadict, labeldict


In [ ]:
disease_directory = '/content/drive/MyDrive/mitdb/mitdb/1.0.0'
healthy_directory = '/content/drive/MyDrive/mitdb/nsrdb/1.0.0'
disease_data = {}
healthy_data = {}

disease_records = [os.path.basename(file).split('.')[0] for file in glob(os.path.join(disease_directory, '*.dat'))]
healthy_records = [os.path.basename(file).split('.')[0] for file in glob(os.path.join(healthy_directory, '*.dat'))]
disease_features = ['MLII', 'V1', 'V2', 'V4', 'V5']
healthy_features = ['ECG1', 'ECG2']

for record_detail in disease_records[:3]:
    record, attributes = dataSaver(record_name=record_detail, directory=disease_directory, features=disease_features)
    disease_data[record_detail] = {"record": record, "attribute": attributes}

for record_detail in healthy_records[:3]:
    record, attributes = dataSaver(record_name=record_detail, directory=healthy_directory, features=healthy_features)
    healthy_data[record_detail] = {"record": record, "attribute": attributes}
# print(data)

MLII V5


100%|██████████| 2559/2559 [00:37<00:00, 68.16it/s]


Data length for MLII: 395
Data length for V5: 395
MLII V1


100%|██████████| 3626/3626 [00:47<00:00, 76.90it/s]


Data length for MLII: 301
Data length for V1: 301
V5 V2


100%|██████████| 2919/2919 [00:40<00:00, 71.51it/s]


Data length for V5: 2099
Data length for V2: 2099
ECG1 ECG2


 42%|████▏     | 23574/56773 [1:57:51<5:21:40,  1.72it/s]

* Reading the data
* Removing all the NaN values, and scaling
* identifying peak
* for each peak, extracting annotations to classify the hearbeat event.
* segmenting the ECG signal around each peak, creating fixed-size windows of data.
* creating one-hot encoded labels based on the annotations
* noise for augmenting and robust (taken help)
* separating data based on ECG lead.

A record can have the following records:

> * p_signal: The physical (float) signal values if physical=True
> * d_signal: The digital (integer) signal values if physical=False
> * fs: Sampling frequency
> * sig_len: Signal length
> * n_sig: Number of signals
> * sig_name: List of signal names
> * units: List of signal units
> * Other metadata like base_date, base_time, etc.

attributes have the following records:

> * sample: An Nx1 integer vector containing the annotation locations in samples relative to the beginning of the record.
> * symbol: An Nx1 character vector describing the annotation types (e.g., 'N' for normal beat, 'V' for premature ventricular contraction, etc.).
> * subtype: An Nx1 integer vector describing annotation subtypes.
> * chan: An Nx1 integer vector indicating the channel associated with each annotation.
> * num: An Nx1 integer vector containing the annotation numbers.
> * aux_note: An Nx1 cell array of strings containing auxiliary information for each annotation.
> * fs: The sampling frequency of the record (if available).


In [ ]:
record_details = os.path.join(disease_directory, '100')
record = wfdb.rdrecord(record_details, smooth_frames = True)
signals0 = preprocessing.scale(np.nan_to_num(record.p_signal[:,0])).tolist()
signals1 = preprocessing.scale(np.nan_to_num(record.p_signal[:,1])).tolist()
peaks, _ = find_peaks(signals0, distance=150)
feature0, feature1 = record.sig_name[0], record.sig_name[1]

In [ ]:
record.sig_name

['MLII', 'V5']

In [ ]:
disease_data['100']['attribute'].sample

In [ ]:
d_record_num = '201'
h_record_num = '16272'

In [ ]:
d_signals = disease_data[d_record_num]['record'].p_signal.T
d_labels = disease_data[d_record_num]['attribute'].symbol
d_fs = disease_data[d_record_num]['record'].fs
h_signals = healthy_data[h_record_num]['record'].p_signal.T
h_labels = healthy_data[h_record_num]['attribute'].symbol
h_fs = healthy_data[h_record_num]['record'].fs

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(d_signals[0][:1000])
plt.title('Disease ECG Signal (First 10000 samples)')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.show()

Amplitude for unhealthy is very less. There is no defining drops. Drop is noisy. Power Spectral might give better idea.

In [ ]:
%pip install PyWavelets

In [ ]:
d_signals.shape

In [ ]:
h_signals.shape

In [ ]:
from scipy import signal

# Power spectral density
f, Pxx = signal.welch(d_signals[1], fs=d_fs)
plt.figure(figsize=(12, 6))
plt.semilogy(f, Pxx)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectral Density [V^2/Hz]')
plt.title('Power Spectral Density of Disease ECG Signal')
plt.grid(True)
plt.show()

Power for healthy is definitely higher. Frequency is also less. This is because the lows in disease is noisy.

In [ ]:
# Power spectral density
f, Pxx = signal.welch(h_signals[1], fs=h_fs)
plt.figure(figsize=(12, 6))
plt.semilogy(f, Pxx)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectral Density [V^2/Hz]')
plt.title('Power Spectral Density of Healthy ECG Signal')
plt.grid(True)
plt.show()

In [ ]:
from scipy.fft import fft, fftfreq

def plot_fft(signal, fs, zf):
    #zf is zoom factor, increase to zoom into, decrease to zoom out
    N = len(signal)
    yf = fft(signal)[:N//zf]
    xf =fftfreq(N, 1.0/fs)[:N//zf]

    plt.figure(figsize=(12, 6))
    plt.semilogy(xf[1:N//4], 2.0/N * np.abs(yf[1:N//4]))
    plt.title('FFT of ECG Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    plt.show()

In [ ]:
plot_fft(signal=d_signals[0], fs=d_fs, zf = 2000)

In [ ]:
from scipy import signal
from matplotlib.colors import PowerNorm
def plot_stft(sig, fs):
    f, t, Zxx = signal.stft(sig, fs, nperseg=256)

    # Compute the magnitude of the STFT
    magnitude = np.abs(Zxx)

    # Calculate the 1st and 99th percentiles for color scaling
    vmin, vmax = np.percentile(magnitude, [5, 95])

    plt.figure(figsize=(12, 6))
    plt.pcolormesh(t, f, magnitude, shading='gouraud',
               norm=PowerNorm(gamma=0.3, vmin=vmin, vmax=vmax),
               cmap='viridis')
    plt.title('STFT of ECG Signal')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.colorbar(label='Magnitude')
    plt.show()

In [ ]:
plot_stft(d_signals[0], d_fs)

In [ ]:
# prompt: Now I want a z score normalization for the arrythmic signals

import numpy as np

def zscore_normalize(signals):
  """
  Performs Z-score normalization on the given signals.

  Args:
    signals: A NumPy array of ECG signals.

  Returns:
    A NumPy array of normalized signals.
  """
  mean = np.mean(signals, axis=0)
  print(mean)
  std = np.std(signals, axis=0)
  print(std)
  normalized_signals = (signals - mean) / std
  return normalized_signals

# Example usage with disease signals (assuming d_signals is your array)


In [ ]:
normalized_d_signals = zscore_normalize(d_signals)

In [ ]:
normalized_h_signals = zscore_normalize(h_signals)


In [ ]:
normalized_d_signals

In [ ]:
normalized_h_signals

In [ ]:
def segment_ecg(signal, window, step):
    segments = []
    for i in range(0, len(signal) - window + 1, step):
        segment = signal[i:i+window]
        segments.append(segment)
    return np.array(segments)

In [ ]:
def label_data(d_windows, h_windows):
    X = np.concatenate((d_windows, h_windows), axis=0)
    y = np.concatenate((np.ones(len(d_windows)), np.zeros(len(h_windows))), axis=0)

    return X, y

In [ ]:
d_signals = disease_data[d_record_num]['record']
h_signals = healthy_data[h_record_num]['record']

In [ ]:
classes = ['N','V','/','A','F','~']#,'L','R',f','j','E','a']#,'J','Q','e','S']
features = ['MLII', 'V1', 'V2', 'V4', 'V5']

In [ ]:
Nclass = len(classes)
datadict, datalabel= dict(), dict()

In [ ]:
for feature in features:
  datadict[feature] = list()
  datalabel[feature] = list()

In [ ]:
!nvidia-smi